In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
df.shape

(8693, 14)

In [5]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
# slice columns of numerical data
numeric_cols = df.select_dtypes(include=np.number)
print(numeric_cols)

       Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck
0     39.0          0.0        0.0           0.0     0.0     0.0
1     24.0        109.0        9.0          25.0   549.0    44.0
2     58.0         43.0     3576.0           0.0  6715.0    49.0
3     33.0          0.0     1283.0         371.0  3329.0   193.0
4     16.0        303.0       70.0         151.0   565.0     2.0
...    ...          ...        ...           ...     ...     ...
8688  41.0          0.0     6819.0           0.0  1643.0    74.0
8689  18.0          0.0        0.0           0.0     0.0     0.0
8690  26.0          0.0        0.0        1872.0     1.0     0.0
8691  32.0          0.0     1049.0           0.0   353.0  3235.0
8692  44.0        126.0     4688.0           0.0     0.0    12.0

[8693 rows x 6 columns]


In [7]:
# impute missing numeric values
from sklearn.impute import SimpleImputer

imp = SimpleImputer()
df[numeric_cols.columns] = imp.fit_transform(df[numeric_cols.columns])

In [8]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [9]:
df.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               81
VIP                2
RoomService     1274
FoodCourt       1508
ShoppingMall    1116
Spa             1328
VRDeck          1307
Name            8473
Transported        2
dtype: int64

In [10]:
#convert Boolean columns to 0 and 1 values
boolean_cols = df.select_dtypes(include='bool')
df[boolean_cols.columns] = df[boolean_cols.columns] * 1

In [11]:
df['VIP'] = df['VIP'] * 1
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   int64  
dtypes: float64(6), int64(1), object(7)
memory usage: 950.9+ KB


In [13]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [14]:
#remove rows with missing values

new_df = df.dropna(axis=0)
new_df.shape

(7559, 14)

In [15]:
new_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [16]:
new_df.nunique()

PassengerId     7559
HomePlanet         3
CryoSleep          2
Cabin           5957
Destination        3
Age               81
VIP                2
RoomService     1167
FoodCourt       1375
ShoppingMall    1051
Spa             1225
VRDeck          1199
Name            7540
Transported        2
dtype: int64

In [17]:
from sklearn.preprocessing import OneHotEncoder

#hot = OneHotEncoder()
#new_df = hot.fit_transform(new_df[['HomePlanet', 'Destination']])

new_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [18]:
enc1 = pd.get_dummies(new_df['HomePlanet'], prefix='HomePlanet')
enc1.head()

enc2 = pd.get_dummies(new_df['Destination'], prefix='Destination')
enc2.head()

print(enc1.columns)

Index(['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars'], dtype='object')


In [19]:
# dropping obsolete columns and inserting one-hot encoded ones

enc_df = pd.concat([new_df, enc1, enc2], axis=1)

enc_df = enc_df.drop(['HomePlanet', 'Destination'], axis=1)

enc_df.head()

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,False,B/0/P,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,0,1,0,0,0,1
1,0002_01,False,F/0/S,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,1,0,0,0,0,1
2,0003_01,False,A/0/S,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,0,1,0,0,0,1
3,0003_02,False,A/0/S,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,0,1,0,0,0,1
4,0004_01,False,F/1/S,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,1,0,0,0,0,1


In [20]:
enc_df = enc_df.drop('Transported', axis=1)
enc_df['Transported'] = df['Transported']
enc_df.head()

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Transported
0,0001_01,False,B/0/P,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1,0,0,0,1,0
1,0002_01,False,F/0/S,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,0,0,0,0,1,1
2,0003_01,False,A/0/S,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,1,0,0,0,1,0
3,0003_02,False,A/0/S,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,1,0,0,0,1,0
4,0004_01,False,F/1/S,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,0,0,0,0,1,1


In [21]:
enc_df.shape

(7559, 18)

In [22]:
enc_df['Cabin'].nunique()

5957

In [23]:
# will drop Cabin and Name features for now

enc_df = enc_df.drop(['Cabin', 'Name'], axis=1)
enc_df.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Transported
0,0001_01,False,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1,0
1,0002_01,False,24.0,0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,1,1
2,0003_01,False,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,1,0
3,0003_02,False,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,1,0
4,0004_01,False,16.0,0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,1,1


In [24]:
from sklearn.model_selection import train_test_split

X = enc_df.iloc[:, 1:-1]
y = enc_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [25]:
# model 1: random forest
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc_model = rfc.fit(X_train, y_train)

# # model 2: logistic regression
# from sklearn.linear_model import LogisticRegression
# lc = LogisticRegression(penalty='None')
# lc_model = lc.fit(X_train, y_train)

# model 2: k nearest neighbour
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

# model 3: SVM
from sklearn.svm import SVC
svc = SVC()
svc_model = svc.fit(X_train, y_train)

# voting classifier
from sklearn.ensemble import VotingClassifier

classifiers = [('rfc', rfc_model), ('knn', knn_model), ('svc', svc_model)]

vc = VotingClassifier(estimators=classifiers, voting='hard')
vc_model = vc.fit(X_train, y_train)

In [26]:
from sklearn.metrics import accuracy_score

y_pred_rfc = rfc_model.predict(X_test)
score_rfc = accuracy_score(y_test, y_pred_rfc)

print("Accuracy score of random forest is", score_rfc)

y_pred_knn = knn_model.predict(X_test)
score_knn = accuracy_score(y_test, y_pred_knn)

print("\n\nAccuracy score of k nearest neighbour is", score_knn)

y_pred_svc = svc_model.predict(X_test)
score_svc = accuracy_score(y_test, y_pred_svc)

print("\n\nAccuracy score of support vector classifier is", score_svc)

y_pred_vc = vc_model.predict(X_test)
score_vc = accuracy_score(y_test, y_pred_vc)

print("\n\nAccuracy score of voting classifier is", score_vc)

Accuracy score of random forest is 0.7777777777777778


Accuracy score of k nearest neighbour is 0.7702821869488536


Accuracy score of support vector classifier is 0.7746913580246914


Accuracy score of voting classifier is 0.7746913580246914


In [27]:
# apply preprocessing to test data

test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [28]:
imp = SimpleImputer()
test[numeric_cols.columns] = imp.fit_transform(test[numeric_cols.columns])

In [29]:
#convert Boolean columns to 0 and 1 values
boolean_cols = test.select_dtypes(include='bool')
test[boolean_cols.columns] = test[boolean_cols.columns] * 1

In [30]:
test['VIP'] = test['VIP'] * 1
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [31]:
print(test.isnull().sum())

test['HomePlanet'] = test['HomePlanet'].fillna(random.choice(['Earth', 'Europa', 'Mars']))
test['CryoSleep'] = test['CryoSleep'].fillna(random.choice([0, 1]))

cabin_vals = test['Cabin'].unique()
test['Cabin'] = test['Cabin'].fillna(random.choice(cabin_vals))

dest_vals = test['Destination'].unique()
test['Destination'] = test['Destination'].fillna(random.choice(dest_vals))

test['VIP'] = test['VIP'].fillna(random.choice([0, 1]))

print(test.isnull().sum())

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age               0
VIP              93
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name             94
dtype: int64
PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin            0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
dtype: int64


In [32]:
#remove rows with missing values

#new_test = test.dropna(axis=0)
new_test = test

In [33]:
enc1 = pd.get_dummies(new_test['HomePlanet'], prefix='HomePlanet')
enc1.head()

enc2 = pd.get_dummies(new_test['Destination'], prefix='Destination')
enc2.head()

,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1


In [34]:
enc_test = pd.concat([new_test, enc1, enc2], axis=1)

enc_test = enc_test.drop(['HomePlanet', 'Destination'], axis=1)

enc_test.head()

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,G/3/S,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,1,0,0,0,0,1
1,0018_01,False,F/4/S,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,1,0,0,0,0,1
2,0019_01,True,C/0/S,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0,1,0,1,0,0
3,0021_01,False,C/1/S,38.0,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0,1,0,0,0,1
4,0023_01,False,F/5/S,20.0,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,1,0,0,0,0,1


In [35]:
enc_test.shape

(4277, 17)

In [36]:
enc_test = enc_test.drop(['Cabin', 'Name'], axis=1)
enc_test.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,27.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,1
1,0018_01,False,19.0,0,0.0,9.0,0.0,2823.0,0.0,1,0,0,0,0,1
2,0019_01,True,31.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0
3,0021_01,False,38.0,0,0.0,6652.0,0.0,181.0,585.0,0,1,0,0,0,1
4,0023_01,False,20.0,0,10.0,0.0,635.0,0.0,0.0,1,0,0,0,0,1


In [37]:
model_predictions = vc_model.predict(enc_test.iloc[:, 1:])

In [38]:
model_predictions = model_predictions.astype(bool)
print(model_predictions)

[ True False  True ...  True  True False]


In [39]:
# print(enc_test.iloc[:, 0])
# print(model_predictions)
#sub1 = pd.DataFrame(enc_test.iloc[:, 0], columns=['PassengerId'])
#sub2 = pd.DataFrame(model_predictions, columns=['Transported'])
#submissions = pd.concat([sub1, sub2], axis=1)
submissions = pd.DataFrame({'PassengerId': test.iloc[:, 0], 'Transported': model_predictions})
submissions.head()

# submissions.shape

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [40]:
submissions.to_csv('submission.csv', index=False)